In [ ]:
#from mf_modules import comfortmodels
%matplotlib widget

import numpy as np
import ipywidgets as widgets
from IPython.display import HTML, display, clear_output
import matplotlib.pyplot as plt
import math as m
import numpy as np
import time

np.set_printoptions(precision=2)

In [ ]:
"""
Reference - Liddament - Guide to Energy Efficient Ventilation
pp. 264

@author: O.Beckett
"""

### AIDA ####

BOXLAYOUT = widgets.Layout(
            display='flex',
            flex_flow='column',
            border='solid 1px grey',
            align_items='stretch',
            margin="2px")

INSTRUCTIONS = "<h2>Guide</h2> \
                <h3>Properties:</h3>\
                <b>Global</b> <br/>\
                <b>Internal Temperature:</b> The temperature of the room - only used when Internal Gain is 0W <br/>\
                <b>External Temperature:</b> The outdoor dr bulb temperature. <br/>\
                <b>Internal Gain:</b> Heat Gain in the room - if greater than zero then this will be used - otherwise the user defined internal temperature is considered. <br/>\
                <b>Wind Speed:</b> The wind speed in m/s at a height of 10m. <br/>\
                <b>Opening</b> <br/>\
                <b>Area:</b> The free area of the opening<br/>\
                <b>Discharge Coefficient:</b> Open windows=0.6, Louvres=0.4, Acoustic Louvres=0.2 <br/>\
                <b>Height:</b> The height above floor level of the opening. <br/>\
                <b>Wind Coeff. / Mech. flowrate (m³/s):</b> The Wind coefficient of the opening (ref DIBSE Guide A 4-8) OR the Mechanical Supply <br/>\
                <b>Mechanical Supply/Extract?:</b> Opening is a mechanical inlet/outlet. If this is checked the Wind coefficient is used as the flow rate in m³/s (+ve is supply, -ve is extract.)  <br/>\
                <h3>Functions:</h3>\
                <b>Add Opening:</b> Add an opening to the room. <br/>\
                <b>Remove Opening:</b> Remove an opening to the room. <br/>\
                <b>Solve:</b> Automatic solving is enabled however it can also be done manually. <br/>\
                <h3>Outputs:</h3>\
                <b>Imbalance:</b> The flow imbalance in the room the smaller the better. This represents the calculation accuracy. <br/>\
                <b>Internal Temperature:</b> If Internal Gain is greater than 0W this is the calculated internal temperature.\
                N.B. When using mechanical supply the supply air is assumed to be outside air temperature. For different supply temperatures add the heating/cooling to the internal gain.\
                This will require a hand calc.<br/>\
                <b>Flowrate:</b> The volume flow rate through each opening (+ve is into the room, -ve is out of the room.) <br/>"

HEADER = widgets.VBox([
widgets.HTML("<h1>Welcome to AIDA</h1>"),
widgets.HTML("<h2>Air Infiltration Development Algorithm</h2>"),
widgets.HTML("M Liddament - AIVC Guide to Ventilation 1995 <br/> Extended by O Beckett 2019"),
widgets.HTML(INSTRUCTIONS)
],
layout=BOXLAYOUT)

In [3]:
D = 1.26 #density of air
CP = 1005

In [4]:
class Opening():

    def __init__(self):
        self.default_area = 1.0
        self.default_height = 1.0 #height of centre
        self.default_cp = 0.0 #wind pressure coefficient
        self.on_change = None
        self.name = ""
        self.flow_rate = 0.0
        #self.controls_container = self.controls
        
    def _create_checkbox(self, label, default=False):
        checkbox_label = widgets.Label(label)
        checkbox = widgets.Checkbox(value=default)
        checkbox.observe(self.on_change, names=['value'])
        checkbox_box = widgets.HBox([checkbox_label, checkbox])
        return checkbox, checkbox_box
        
    def _create_value_slider(self, min_val, max_val, label, default=1, step=0.1):
        slider_label = widgets.Label(label)
        slider = widgets.FloatSlider(
            value=default,
            min=min_val, max=max_val, step=step,
            layout=widgets.Layout(width='300px')
        )
        slider.observe(self.on_change, names=['value'])
        slider_box = widgets.HBox([slider_label, slider])
        return slider, slider_box
        
    @property
    def C(self):
        if self.boolMech: return self.cp
        return self.default_C()
        
    @property
    def area(self):
        return self.area_slider.value
    
    @property
    def cd(self):
        return self.cd_slider.value
    
    @property
    def cp(self):
        return self.cp_slider.value
    
    @property
    def exponent(self):
        if self.boolMech: return 0.0
        return 0.5
        
    @property
    def boolMech(self):
        return self.mech_cb.value
    
    @property
    def height(self):
        return self.height_slider.value
    
    @property
    def controls(self):
        try:
            return self._controls_container
        except:
            self.area_slider, area_sliderbox = self._create_value_slider(0, 25, "Area (m²)", default=self.default_area)
            self.cd_slider, cd_sliderbox = self._create_value_slider(0, 1, "Discharge Coefficient", default=0.6, step=0.01)
            self.height_slider, height_sliderbox = self._create_value_slider(0, 25, "Height (m)", default=self.default_height)
            self.cp_slider, self.cp_sliderbox = self._create_value_slider(-10, 10, "Wind Coeff. / Mech. flowrate (m³/s)", default=self.default_cp)
            self.mech_cb, mech_cbbox = self._create_checkbox("Mechanical Supply/Extract?", default=False)
            self.flow_lbl = widgets.HTML("<h3>Flowrate (m³/s) -</h3>")
           
            self._controls_container = widgets.VBox([
                widgets.HTML("<b>Opening {0}</b>".format(self.name)),
                area_sliderbox,
                cd_sliderbox,
                height_sliderbox,
                self.cp_sliderbox,
                mech_cbbox,
                self.flow_lbl],
                layout=BOXLAYOUT)   
            
            return self._controls_container

    @classmethod
    def from_params(cls, area, height, cp, on_change, C=None, name=""):
        result = cls()
        result.default_area = area
        result.default_height = height
        result.default_cp = cp
        result.user_C = C
        result.on_change = on_change
        result.name = name
        return result

    def default_C(self):
        return self.cd*self.area*m.sqrt(2./D)

In [5]:
class AIDA():
    '''
    Air Infiltration Development Algorithm
    '''

    def __init__(self):
        self.openings = []
        self.openings.append(Opening.from_params(3.8, 8, 0.0, self.solve, name="1"))
        self.openings.append(Opening.from_params(3.8, 9, 0.0, self.solve, name="2"))
  
        self.vb = widgets.VBox([])
        self.calc_controls_box = widgets.VBox([], layout=BOXLAYOUT)
        
        self.fig, (hax) = plt.subplots(1, 1, figsize=(12, 6))
        
    @property
    def L(self):
        return len(self.openings)

    @property
    def external_temp(self):
        return self.ext_temp_slider.value
    
    @property
    def internal_temp(self):
        return self.int_temp_slider.value
    
    @property
    def heat_gain(self):
        return self.heat_gain_slider.value
    
    @property
    def wind_speed(self):
        return self.wind_slider.value
    
    def add_meshed_opening(self, totalA, num, h1, height): #Currently unused and incorrect in the object orientated world.
        self.L += num
        for i in range(num):
            self.H.append(h1 + (i+0.5)*height/num)
            self.P.append(0)
            self.N.append(0.5)
            self.C.append(0.4*totalA*m.sqrt(2./D)/num)

    def on_bttn_clicked(self, b): 
        new_name = str(len(self.openings) + 1)
        self.openings.append(Opening.from_params(3.8, 8, 0.0, self.solve, name=new_name))
        self.update_display()

    def on_rm_bttn_clicked(self, b):
        self.openings = self.openings[:-1]
        self.update_display()
            
    def _create_value_slider(self, min_val, max_val, label, default=1):
        slider_label = widgets.Label(label)
        slider = widgets.FloatSlider(value=default, min=min_val, max=max_val, step=0.1, layout=widgets.Layout(width='300px'))
        slider.observe(self.solve, names=['value'])
        slider_box = widgets.HBox([slider_label, slider])
        return slider, slider_box
            
    def calc_controls(self):
        self.btn = widgets.Button(description = 'Add Opening') 
        self.btn_rm = widgets.Button(description = 'Remove Opening') 
        self.btn_solve = widgets.Button(description = "Solve")
        self.lbl_results = widgets.HTML("<h3>Flowrates - </h3>")
        self.lbl_balance = widgets.HTML("<h3>Imbalance - </h3>")
        self.lbl_internal_temp = widgets.HTML("<h3>Internal Temperature - </h3>")
        
        self.btn.on_click(self.on_bttn_clicked)
        self.btn_rm.on_click(self.on_rm_bttn_clicked)
        self.btn_solve.on_click(self.solve)
        
        self.int_temp_slider, int_temp_slider_box = self._create_value_slider(-10, 40, "Internal Temperature (°C)", default=24)
        self.ext_temp_slider, ext_temp_slider_box = self._create_value_slider(-10, 40, "External Temperature (°C)", default=20)
        self.heat_gain_slider, heat_gain_slider_box = self._create_value_slider(0, 10000, "Internal Gain (W)", default=0)
        self.wind_slider, wind_slider_box = self._create_value_slider(0, 20, "Wind Speed (m/s)", default=1)
        
        button_group = widgets.HBox([self.btn, self.btn_rm, self.btn_solve])
        weather_group = widgets.VBox([int_temp_slider_box, ext_temp_slider_box, heat_gain_slider_box, wind_slider_box])
        results_group = widgets.VBox([self.lbl_results, self.lbl_balance, self.lbl_internal_temp])
        
        return [button_group, weather_group, results_group]
    
    def display(self):
        display(self.app)
        
    def update_display(self):
        self.controls = [o.controls for o in self.openings]
        self.output = widgets.Output()
        
        self.calc_controls_box.children = tuple(self.calc_controls())
        self.vb.children = tuple(self.controls)
        self.app = widgets.VBox([
            widgets.HBox([
            self.calc_controls_box,
            self.vb, self.output]),
            HEADER
        ])
       
    def calc_T(self, H, I, E, W):
        '''Returns the opening pressures'''
        S = -3455*H*((1/(E+273.15))-(1/(I+273.15)))
        T = W + S
        return T
        
    def solve(self, sender):
        L = self.L
        H = np.array([x.height for x in self.openings])
        P = np.array([x.cp for x in self.openings])
        N = np.array([x.exponent for x in self.openings])
        C = np.array([x.C for x in self.openings])
        
        boolMech = N==0
        HEATGAIN = self.heat_gain
        E = self.external_temp
        I = self.internal_temp
        U = self.wind_speed

        ###Pressure Calculation###
        W = 0.5*D*U*U*P
        T = self.calc_T(H, I, E, W)
        
        #Initialise temperature difference between iterations
        delT = 1000
        
        while(delT>0.01):
            
            ### Calculate Flows and refresh for each temperature iteration###
            R = -100. #Internal Pressure
            X = 3. #Iteration pressure step
            Y = 0 #Iteration counter
            B = -1
            F = np.zeros(L) #Calculated flow rate
        
            while((B<=0 or B/np.sum(abs(F))>0.01) and Y<20000):
                if Y%1000==0: self._update_result_label(F, I, sum(F))
                Y+=1
                B = 0 #Flow balance

                R = R+X
                T[boolMech] = R
                O = T - R #Pressure difference across each flow path.
                divisor = abs(O)
                numerator = O
                numerator[divisor==0]=1
                divisor[divisor==0]=1
                F = np.nan_to_num(C*abs(O)**N*O/divisor)
                B = np.sum(F)

                if B<0:
                    R = R-X
                    X = X*0.99
                    if Y%100==0: X=100/Y
                else:
                    pass

            self.F = F #Flowrates
            self.Q = sum(F) #Total flow rate or imbalance
            self.QT = sum(F[F>0]) #Net flow rate
            
            if(HEATGAIN>0):
                #Adjust the internal temperature if using heat gain, otherwise force exit after one iteration.
                I_OLD = I
                I_NEW = E + (HEATGAIN/(self.QT*D*CP))
                I = 0.7*I + 0.3*I_NEW #0.7 relaxation factor.
                T = self.calc_T(H, I, E, W)
                delT = abs(I-I_OLD)
            else:
                delT=0
                
            self._update_result_label(self.F, I, delT, self.Q)

        self.plot_graph()
    
    def perc(self, num, denom):
        return 100*num/(denom+0.00001)
            
    def _update_result_label(self, F, I, delT, balance=0):
        self.lbl_results.value = "<h3>Flowrates (m³/s) {0}</h3>".format(F)
        self.lbl_balance.value = "<h3>Imbalance {0:.2f}(m³/s); {1:.1f}(%) </h3>".format(balance, self.perc(balance, (np.max(F))))
        self.lbl_internal_temp.value = "<h3>Internal Temperature {0:.1f}(°C)</h3>".format(I, self.perc(delT, I))
        try:
            for i, f in enumerate(F):
                self.openings[i].flow_rate = f
                self.openings[i].flow_lbl.value = "<h3>Flowrate (m³/s) {0:.2f}</h3>".format(f)
        except:
            pass
        
    def _ipython_display_(self):
        self.update_display()
        self.display()
        
    def plot_graph(self, sender=None):
        MARKER = "<"
        MARKERSIZE = 20
        GREEN = (0.8, 1.0, 0.2, 0.7)
        RED = 'red'
        BLUE = 'blue'
        t = np.array([x.height for x in self.openings])
        s = np.array([x.flow_rate for x in self.openings])
        mech = np.array([x.boolMech for x in self.openings])

        plt.cla()
        plt.plot(s[s<0], t[s<0], MARKER, markersize=MARKERSIZE, color=RED) #Extract
        plt.plot(np.array([0]*len(t[s<0])), t[s<0], MARKER, markersize=MARKERSIZE, color=RED) #Extract
        plt.plot(s[s>0], t[s>0], MARKER, markersize=MARKERSIZE, color=BLUE)
        plt.plot(np.array([0]*len(t[s>0])), t[s>0], MARKER, markersize=MARKERSIZE, color=BLUE) #Incoming in blue
        plt.plot(np.array([0]*len(t[mech])), t[mech], MARKER, markersize=MARKERSIZE, color=GREEN)#Mechanical in green
        plt.plot(s[mech], t[mech], MARKER, markersize=MARKERSIZE, color=GREEN) #Mechanical in green
        plt.hlines(t, [0], s, lw=2)
        plt.set_xlabel('Flow Rate (m³/s)')
        plt.set_ylabel('Height (m)')
        plt.set_title('Flow Rates')

        with self.output:
            display(self.fig)
            plt.show(self.fig)

        

In [6]:
from ipyrun._ipydisplayfile import DisplayFile
#help(DisplayFile)


In [9]:
from mf_modules.datamine_functions import recursive_glob
#help(recursive_glob)
recursive_glob(rootdir='..\data\processed', pattern='*', recursive=True)
fpth = '..\\data\\processed\\gbxml_datagrab.xlsx'
d = DisplayFile(fpth)
#d.preview_fpth()

In [10]:
from mf_modules.eng_calcs import vent_cooling
help(vent_cooling)
vent_cooling(no_ppl=100,v_m3=378.571322)

Help on function vent_cooling in module mf_modules.eng_calcs:

vent_cooling(no_ppl=5, v_m3=80, deltaT=3, ppl_gain_sens=75, ppl_gain_lat=55, other_gain_w=0, show=True)
    cooling associated to a vent flowrate as a function of gains and deltaT
    
    can be used with AIDA to size windows / nat vent requirements
    Note.
    from mf_modules.import_notebook import *
    from notebooks.NaturalVentCalculator import *
    AIDA()
    
    Args:
        no_ppl (int): =5 no,  number of people
        v_m3 (float): =80 m3, volume of room
        deltaT (float): =3 degC, temperature differential between room T and supply air T
        ppl_gain_sens (float): =75 w/person, sensible heat gain to room from people
        ppl_gain_lat (float): =55 w/person, latent heat gain to room from people
        other_gain_w (float): =0 w, other gains to the room
        show (bool): =True, show a formatted output of results
    
    Returns:
        v_flow (float): m3/s, volume flow rate of ventilation for c

<b>total number of people</b> = 100 ppl

<b>assumed people latent gain</b> = 55 w/p

<b>assumed people sensible gain</b> = 75 w/p

<b>total heat gain from people</b> = 13.0 kw

<b>required flowrate to overcome gains</b> = 0.84 m3/s

0.84

In [11]:
a = AIDA()
a

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [1]:
def vent_cooling(no_ppl=5, v_m3=80, deltaT=3, ppl_gain_sens=75, ppl_gain_lat=55, other_gain_w=0, show=True):
    '''
    cooling associated to a vent flowrate as a function of gains and deltaT
    
    can be used with AIDA to size windows / nat vent requirements
    Note.
    from mf_modules.import_notebook import *
    from notebooks.NaturalVentCalculator import *
    AIDA()

    Args:
        no_ppl (int): =5 no,  number of people
        v_m3 (float): =80 m3, volume of room
        deltaT (float): =3 degC, temperature differential between room T and supply air T
        ppl_gain_sens (float): =75 w/person, sensible heat gain to room from people
        ppl_gain_lat (float): =55 w/person, latent heat gain to room from people
        other_gain_w (float): =0 w, other gains to the room
        show (bool): =True, show a formatted output of results

    Returns:
        v_flow (float): m3/s, volume flow rate of ventilation for cooling

    '''
    
    ppl_gain = no_ppl*(ppl_gain_sens+ppl_gain_lat)
    ttl_sens = no_ppl * ppl_gain_sens
    ttl_lat = no_ppl * ppl_gain_lat
    ttl_gain = ttl_lat + ttl_sens
    #ttl_gain = np.round(ttl_gain/1000,2)
    Cp = 4200 #kj/k.kg
    ro = 1.225 #kg/m3

    #m = ro * v_m3 #kg
    #e = m * Cp * deltaT #kj
    v_flow = np.round(ttl_gain/(ro * Cp * deltaT),2)#m3
    
    if show==True:
        display(Markdown('<b>total number of people</b> = {0} ppl'.format(no_ppl)))
        display(Markdown('<b>assumed people latent gain</b> = {0} w/p'.format(ppl_gain_lat)))
        display(Markdown('<b>assumed people sensible gain</b> = {0} w/p'.format(ppl_gain_sens)))
        if other_gain_w==0:
            display(Markdown('<b>total heat gain from people</b> = {0} kw'.format(ttl_gain)))
        else:
            display(Markdown('<b>total heat gain from people</b> = {0} w'.format(ppl_gain)))
            display(Markdown('<b>total other heat gain</b> = {0} w'.format(other_gain_w)))
        display(Markdown('<b>required flowrate to overcome gains</b> = {0} m3/s'.format(v_flow)))
    
    return v_flow

In [5]:
def vent_cooling(no_ppl=5, v_m3=80, deltaT=3, ppl_gain_sens=75, ppl_gain_lat=55, other_gain_w=0, show=True):
    inputs = [
        {
        'name':'no_ppl',
        'value':no_ppl,
        'label':'no,  number of people'
        },
        {
        'name':'v_m3',
        'value':v_m3,
        'label':'m3, volume of room'
        },
        {
        'name':'deltaT',
        'value':deltaT,
        'label':'degC, temperature differential between room T and supply air T'
        },
        {
        'name':'ppl_gain_sens',
        'value':ppl_gain_sens,
        'label':'w/person, sensible heat gain to room from people'
        },
        {
        'name':'ppl_gain_lat',
        'value':ppl_gain_lat,
        'label':'w/person, latent heat gain to room from people'
        },
        {
        'name':'other_gain_w',
        'value':other_gain_w,
        'label':'w, other gains to the room'
        }
    ]
    
    outputs = [
        {
        'name':'v_flow',
        'value':v_flow,
        'label':'m3/s, volume flow rate of ventilation for cooling'
        },
    ]

    return pd.DataFrame.from_dict(di)
import pandas as pd
vent_cooling()

,name,value,label
0,no_ppl,5,"no, number of people"
1,v_m3,80,"m3, volume of room"
2,deltaT,3,"degC, temperature differential between room T ..."
3,ppl_gain_sens,75,"w/person, sensible heat gain to room from people"
4,ppl_gain_lat,55,"w/person, latent heat gain to room from people"
5,other_gain_w,0,"w, other gains to the room"
